First I will import a shitload of poliastro thingies, they are not all truly neccesary. Notice the CtllDes import, the core and utils elements

In [1]:
import CtllDes 
from CtllDes.core import ctll, satellite

In [2]:
### Specify Instruments ###
from CtllDes.core import instrument

cam1 = instrument.Camera(0,0)
cam2 = instrument.Camera(0,1)
cam3 = instrument.Camera(0,3)


### Specify Physical Specifications ###
from CtllDes.core import specs

drag_coef = 1
m1 = 100 #kg
m2 = 80 #kg
m3 = 200 #kg

A = 100 #m2

spec1 = specs.Specifications(m=m1,Cd=drag_coef,A_over_m=A)
spec2 = specs.Specifications(m=m1,Cd=drag_coef,A_over_m=A)
spec3 = specs.Specifications(m=m3,Cd=drag_coef,A_over_m=A)



The utility function <strong>helio_geo_synchronous</strong> receives an integer, and returns a dataframe with all the possible circular orbit classic elements that are helio and geo synchronous with a D days period. It is pretty straigthforward to transform the desired filtered row into a classical State.

In [ ]:
from CtllDes.utils import helio_geo_synchronous

D = 7
orb_df = helio_geo_synchronous(D)

Lets say that you want to filter out heights. One easy way to filter out whatever state you may want to generalize uses pandas loc . If you are not familiar with pandas, read the basic examples in their docs.

In [4]:
filtered = orb_df.loc[(df['p'] < 900) & (df['p'] > 500)]
filtered


Check if you got any filtered orbits and pick. Notice that <strong>raan</strong> and <strong>nu</strong> will be set to 0. Feel free to change it to make it pass over the desired target at least one time.

In [ ]:
st1 = filtered[0]

orb1 = Orbit.from_classical(Earth,
                            st1['a'],
                            st1['ecc'],
                            st1['inc'],
                            st1['raan'],
                            st1['argp'],
                            st1['nu'],
                            epoch=J200,
                            plane=Planes.EARTH_EQUATOR)

Now its about time to build the satellite using the <strong>from_orbit </strong> classmethod. 

In [ ]:
sat1 = satellite.from_orbit(orb1,spec=spec1,instruments=cam1)

Lets keep up building the rest of the satellites with different methods. Building from a specific orbit was already covered, also adding the use of some util function that defines orbits. We could use as well for any orbit something like the following

In [ ]:
### Proof of Concept ###
sat_poc = satellite.from_orbit(Orbit.helio_synchronous) 

#if instruments are not defined, they will be an empty list
print(sat.instruments)

#if specs are not defined, DefaultSpec will be used
print(sat.specs)
print(specs.DefaultSpec)

#if status is not defined, it will be set to online
print(sat.status)



Now, proven the ways you can build your sat, lets grab the two satellites and put them right into a TPF WalkerDelta constellation. The ways of getting synchronous or heliosynchronous orbits can be achieved by using the <strong>poliastro</strong> Orbit classmethods, and then, as showed in the example above this cell, you can build <strong>from_orbit</strong>. Imagine that we already now the classical orbit parameters for this constellation, being:


In [ ]:
from astropy import units as u

p = 35000 * u.km
ecc = 0 * u.one
inc = 0 * u.rad
raan = 0 * u.rad
argp = 0 * u.rad
nu = 0 * u.rad
plane = Planes.EARTH_EQUATOR 

Astropy units allow you to manage input into the poliastro package without worrying to much about the unit specifically used. If you find yourself with the desired inclination in degrees, just enter <strong>u.deg</strong> instead of <strong>u.rad</strong>. Then you can directly build a Walker Delta constellation 

In [ ]:
constellation = ctll.Ctll.from_WalkerDelta(T,P,F,p,ecc,inc,argp)

Notice that <strong>raan</strong> and <strong>nu</strong> are not specified, but they can be, as mentioned for the other orbit mentioned before. We should specify the instruments and specifications of the satellites, as well as the TPF pattern.  

In [ ]:
T=2
P=1
F=0

speks = [spec2, spec3]
instr = [cam2, cam3]

constellation = ctll.Ctll.from_WalkerDelta(T,P,F,p,ecc,inc,argp,raan_offset=40*u.deg,nu_offset=180*u.deg,specs=speks,instrumentss=instr)

After building this you may add <strong>sat1</strong> to the constellation in different ways.

In [ ]:
### Directly adding the sat1 ###
constellation.add_sat(sat1)

### Creating 2 constellations and adding them ###
constellation = ctll.Ctll.from_sats([sat1]+constellation.sats)

### Using the add operator (silly example though) ###
constellation_1 = ctll.Ctll.from_sats(sat1)
constellation_2 = constellation #just for clarity
constellation = constellation_1 + constellation_2


Now that you have the constellation up and running, a propagation can be made. Let's do 10 days of simulation with an interval of 1 second, a bunch of points indeed.

In [ ]:
rvs = constellation.rv(10)

If you read the docs you will notice that <strong>rv</strong> method returns a list with length equal to the amount of sats, each one reporting the rv to the <strong>constellation.sats</strong> list of satellite. If you want to get the specific Ids of this satellites that are being reported you should check <strong>constellation.online_id</strong>

In [ ]:
ids = constellation.online_id

So rvs will be a list of tuples (rr,vv), each one of these being quantity objects arrays of size $\frac{T\cdot 3600 \cdot 24}{dt}$. Therefore if you want to extract and plot the orbit lets say yoy may do this by using matplotlib or using the default constellation plot, that is specially and beautifully taylor made to display awesome graphixz (sike, no that great, just cartopy default plotting). The library is data oriented, it is not self embedded the analysis of the data, that corresponds to the analytical strings that will be explained alter on on this notebook.

In [5]:
from CtllDes.core.utils import plot_ctll_orbit, plot_ctll_gt
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
last_sat_r = rvs[-1][0]

### how you are supposed to do it
fig1 = plt.figure()
ax = fig1.add_subplot(111,projection='3d')
ax.scatter(last_sat_r[:,0], last_sat_r[:,1], last_sat_r[:,2],s=0.5,c='k') #xyz obviously

### embedded
utils.plot_ctll(rvs)

plt.show()

Very similar to this you may find the subsatellite points. In a very equal fashion, you should use the data, but for you CtllDes lovers, Ill give you some embedded crappy groundtrack plot functionality. Here you go

In [7]:
ssps = constellation.ssps(1)

### how you are supposed to do it 
first_sat_ssps = ssps[0]
first_sat_lon = first_sat_ssps[0][0]
first_sat_lat = first_sat_ssps[0][1]

fig = plt.figure()
plt.scatter(first_sat_lon,first_sat_lat,c='k',s=0.5)

### embedded
fig2 = plot_ctll_gt(ssps)

######
plt.show()

Now that the plotting is done, you gpu thanks you. Afterwards you may want to do some nice coverage analysis. In order to do this you may have to use the Target module, since the cool methods will only recognize my special and stupid ass class called targets, just wait to see the full path to the targets folders, you will trip your tits out.

In [ ]:
from CtllDes.targets.targets import Targets, Target

so many targets.... But there is a positive side to it, it lets you build the coverage points of sampling from country or states just like described in the cell below. 

In [9]:
tgts = Targets.from_country('Argentina',N=10) #N is approximately 0.6 the square of the number of total sample points, more on this later
fig = tgts.plot()
plt.show()

So lets break it down. First you are building a Target container, correctly named Targets, from a country, just specify the name or ISO code and if not found and error will pop up, don't worry. So what this is actually doing, is sampling inside the territory of the specified country correcting the number of points depending on latitude. This is done in 4 simple steps.
<ul>
    <li> Check if Country exists, and found the minimum box of coordinates that surround it.
    <li> Generate a grid of equally N spaced points in latitude and distribute in $N\cdot \cos(latitude)$ points in longitude. This is done in order to dont get overestimations near the poles (imagine what would happen if you distributed uniformly on a mercator projection, forgeting about the sphericity of earth).
     <li> Filter those coordinates generated checking if they lay inside the selected country.
     </ul>
 
The procedure for states is the same. The only difference is that you have more RegEx tu pick in order to select the state. 

In [ ]:
tgts = Targets.from_state('Buenos Aires')
fig = tgts.plot()
plt.show()

After doing this you may want to do a coverage analysis, only if your constellation has coverage instruments on it. This is the case for this example so you could build a Coverages object from the classmethod functions. This class is actually a container for the individual Coverage object. You must imagine a Coverage object consisting of a target, a time of propagation, an interval dt of time of integration, and a bunch of merit figures automatically calculated, oriented to assist the design process, checking stuff like revisit time, mean gaps, max gaps, response time etc. If you want more information on this just check Chapter 9 of O.C.D.M. from James R. Wertz. Link in bio

<p style="text-align:center">
    <a href="https://www.amazon.es/Constellation-Design-Management-Technology-Library/dp/1881883078">
        <img src="https://images-na.ssl-images-amazon.com/images/I/41Ca0XLUv6L._SX303_BO1,204,203,200_.jpg">
    </a>
</p>

In [ ]:
from CtllDes.requests.coverage import Coverages

covs = Coverages.from_ctll(constellation,tgts,5)

Now you want to grab the dataframe associated to these data. Remember that there will be $ \# targets \cdot \# coverageInstruments$ points. Once you got the data frame, feel free to do some cool graphix, such as the heatmap described below, with the help of <strong>cartopy</strong> and <strong>ccrs</strong>.

In [ ]:
coverage_df = covs.to_df()
df

Now you have the coverages targets repeated #satellites times. So the user must filter out in terms of the desired output, if you want to collapse the coverages from the whole constellation, feel free to do so from the dataframe. But be aware of the fact that the merit figures dont form a linear vector space.

So if you want to really collapse and create a new type of coverage you may use Coverages methods. When I say collapse I'm refering to the consideration of just one of something, for example you may collapse the satellites coverage, and end up with just #targets amount of merit figures, since the coverage figures are computed for the whole constellation. 

In [ ]:
col_covs = covs.collapse_sats() #If no argument is specified, collapse all satellites

Now with these new figures of merit that represent the whole constellation, we can produce some output, a heatmap parexample.

In [ ]:
### Collapsed dataframe
col_coverage_df = col_covs.to_df()


In [ ]:
### Targets longitude and latitude
tgt_latlons = pd.DataFrame(col_coverage_df['Target'].apply(pd.Series))
tgt_lon = tgt_latlons[0]
tgt_lat = tgt_latlos[1]

The following generated heatmap corresponds to the accumulated time of sight. This does not represent the percentage of coverage, but as you can imagine is very easy to compute 

In [ ]:
import cartopy.crs as ccrs

### Integral of time in view of targets
accum = col_coverage_df['accumulated'].values #feel free to do a nice heatmap of any of the 6 merit figures.

##total seconds of coverage
#Tsecs=col_coverag_df['T']*3600*24
#pofcov = accum/Tsecs

X,Y = np.meshgrid(lon,lats)
fig = plt.figure 
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
plt.pcolormesh(X,Y,accum,cmap='magma') #you may try pofcov as well
plt.colorbar()

plt.show()


In [ ]:

from poliastro.twobody import states as st
from poliastro.bodies import Earth, Mars, Sun
from poliastro.frames import Planes
from poliastro.twobody import propagation,Orbit
